<a href="https://colab.research.google.com/github/yashkumarkalyan/ailawllm/blob/main/Copy_of_ailaw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests beautifulsoup4


In [2]:
import requests
from bs4 import BeautifulSoup

url = "https://www.whitecase.com/insight-our-thinking/ai-watch-global-regulatory-tracker"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Extract text and URLs
text_data = []
for paragraph in soup.find_all("p"):
    text_data.append(paragraph.get_text())

urls = []
for link in soup.find_all("a", href=True):
    urls.append(link['href'])

# Combine text data and URLs
combined_data = {
    "text": text_data,
    "urls": urls
}

print(combined_data)


{'text': ['\nKeeping track of AI regulatory developments around the world.\n\n', 'Keeping track of AI regulatory developments around the world.', 'Artificial intelligence (AI) has made enormous strides in recent years and has increasingly moved into the public consciousness.', "Increases in computational power, coupled with advances in machine learning, have fueled the rapid rise of AI. This has brought enormous opportunities, as new AI applications have given rise to new ways of doing business. It has also brought potential risks, from unintended impacts on individuals (e.g., AI errors harming an individual's credit score or public reputation) to the risk of misuse of AI by malicious third parties (e.g., by manipulating AI systems to produce inaccurate or misleading output, or by using AI to create deepfakes).", 'Governments and regulatory bodies around the world have had to act quickly to try to ensure that their regulatory frameworks do not become obsolete. In addition, internationa

In [3]:
import re

def preprocess_text(text):
    # Remove special characters and numbers
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\d', ' ', text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

processed_texts = [preprocess_text(text) for text in text_data]


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=100)
X = vectorizer.fit_transform(processed_texts)
keywords = vectorizer.get_feature_names_out()

print(keywords)


['act' 'actively' 'ai' 'an' 'and' 'approach' 'approaches' 'are' 'around'
 'as' 'at' 'businesses' 'but' 'by' 'compliance' 'core' 'development'
 'developments' 'different' 'do' 'draft' 'each' 'encouraging' 'engages'
 'enormous' 'ensure' 'ethical' 'eu' 'europe' 'existing' 'expected' 'face'
 'federal' 'first' 'for' 'frameworks' 'from' 'global' 'governance' 'guide'
 'guidelines' 'has' 'have' 'human' 'in' 'inconsistent' 'initiatives'
 'innovation' 'international' 'into' 'introduce' 'is' 'issues' 'it'
 'jurisdictions' 'law' 'laws' 'member' 'national' 'new' 'of' 'on' 'or'
 'policy' 'potential' 'practice' 'principles' 'process' 'proposal'
 'provide' 'provides' 'public' 'published' 'regulate' 'regulation'
 'regulations' 'regulatory' 'responsible' 'rights' 'rise' 'risk' 'risks'
 'safe' 'sector' 'sectors' 'specific' 'states' 'strategy' 'taken' 'that'
 'the' 'these' 'they' 'this' 'to' 'trustworthy' 'will' 'with' 'world'
 'yet']


In [7]:
import pandas as pd

# Determine the shortest length among the lists
min_length = min(len(processed_texts), len(urls))

df = pd.DataFrame({
    "text": processed_texts[:min_length],  # Use slicing to ensure equal lengths
    "urls": urls[:min_length]
})

df.to_csv("fine_tuning_data.csv", index=False)

In [10]:
!pip install datasets transformers
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset

# Load pre-trained model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# Tokenize data
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Use the datasets library's map function instead of pandas
tokenized_datasets = Dataset.from_pandas(df).map(tokenize_function, batched=True)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"]
)

# Train model
trainer.train()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


AttributeError: module 'pyarrow.lib' has no attribute 'ListViewType'